## Example for phenol atomic positions optimization (in a box) using Conquest

In [1]:
import os
import shutil
import subprocess
import matplotlib.pyplot as plt

from distutils.spawn import find_executable
from numpy import amax, amin, matmul, exp, pi, array, zeros, sqrt

from ase.calculators.conquest import Conquest
from ase.units import Rydberg
from ase.build import bulk
from ase.dft.bandgap import bandgap
from ase.io import read

from ase import Atoms
from ase.spacegroup import crystal

from cq_ase_external_lib import print_struct_data
from cq_ase_external_lib import print_occ, get_gapwind
from cq_ase_external_lib import cq_postproc_xsf, cq_print_output_data
from cq_ase_external_lib import read_conquest, conquest_orthorhombic_check

In [2]:
%%bash 
ase --version

ase-3.23.0b1


#### Check if visualisation tools are installed

In [3]:
# Add exe name 
cmd_vis = {'xcrysden' : False}

for cmd, state in cmd_vis.items():
    if ( not shutil.which(cmd) ):
        print('{} not found'.format(cmd))
    else:
        print('{} found'.format(cmd))   
        cmd_vis[cmd] = True
        
# For MacOSX add the path of the name of the app
cmd_app = {'VESTA'      : '/Applications/VESTA/VESTA.app/Contents/MacOS/VESTA',
           'VMDLauncher': '/Applications/VMD_1.9.2.app/Contents/MacOS/VMDLauncher'}

for cmd, path in cmd_app.items():
    if ( not find_executable(path) ):
        print('{} not found'.format(cmd))
    else:
        print('{} found'.format(cmd))   

xcrysden found
VESTA found
VMDLauncher not found


#### Define Conquest environment

In [4]:
CQ_ROOT = '/Users/lioneltruflandier/CONQUEST-develop'
os.environ['ASE_CONQUEST_COMMAND'] = 'mpirun -np 4 '+CQ_ROOT+'/src/Conquest'
os.environ['CQ_PP_PATH'] = CQ_ROOT+'/pseudo-and-pao/'
os.environ['CQ_GEN_BASIS_CMD'] = CQ_ROOT+'/tools/BasisGeneration/MakeIonFiles'

#### Directory for storing calculation files

In [5]:
working_directory = 'cq_example_phenol_optpos'

# Test if `working_directory` exists ? If not create it
if ( not os.path.isdir(working_directory) ):
    os.makedirs(working_directory)

####  Build ASE `Atoms` for C$_6$H$_{5}$OH in a box

- Load structure of C$_6$H$_{5}$OH from `PDB` file 

In [6]:
struct = read('./struct/phenol.pdb')

print('n atoms = {} ?'.format(len(struct)))

n atoms = 13 ?


- Cell parameters in Ang. and deg. [a, b, c, alpha, beta, gamma]. Cell size should be cubic $8\times8\times8$ Ang.

In [7]:
a = 10.0
b = a
c = a
alpha = 90.0
beta  = 90.0
gamma = 90.0

cell = [a,b,c,alpha,beta,gamma]

- Generate the `Atoms` object and store in `struct` 

In [8]:
struct = Atoms(struct,cell=cell,pbc=[True,True,True])

# Conquest can only handle orthorhombic cells
struct = conquest_orthorhombic_check(struct,verbose=True)

- Extract and print main structural data from input structure `struct`

In [9]:
print_struct_data(struct,verbose=1)

Cartesian atomic positions (Ang.)
  C       0.000000      0.000000      0.000000
  C       0.000000      0.000000      1.400000
  C       1.212000      0.000000      2.100000
  C       2.425000      0.000000      1.400000
  C       2.425000      0.000000      0.000000
  C       1.212000      0.000000     -0.700000
  H      -0.943000      0.000000      1.944000
  H       1.212000      0.000000      3.189000
  H       3.368000      0.000000      1.944000
  H       3.368000      0.000000     -0.545000
  H       1.212000      0.000000     -1.789000
  O      -1.224000     -0.021000     -0.679000
  H      -1.067000     -0.018000     -1.615000

Fractional atomic positions (Adim.)
  C       0.000000      0.000000      0.000000
  C       0.000000      0.000000      0.140000
  C       0.121200      0.000000      0.210000
  C       0.242500      0.000000      0.140000
  C       0.242500      0.000000      0.000000
  C       0.121200      0.000000     -0.070000
  H      -0.094300      0.000000    

/Users/lioneltruflandier/Conquest-ase/ase-Conquest/ase/utils/__init__.py:62: FutureWarning: Please use atoms.cell.cellpar() instead
  warnings.warn(warning)


- Save input `struct` in CIF and XSF format 

In [10]:
struct.write(working_directory+'/input.xsf')
struct.write(working_directory+'/input.cif')

- run `VESTA` to check input structure (if possible)

In [11]:
if ( 'VESTA' in cmd_app ):
    subprocess.run([cmd_app['VESTA'], working_directory+'/'+'input.cif'])

2023-07-24 18:31:10.379 VESTA[74288:2929232] It's not legal to call -layoutSubtreeIfNeeded on a view which is already being laid out.  If you are implementing the view's -layout method, you can call -[super layout] instead. Break on void _NSDetectedLayoutRecursion(void) to debug.  This will be logged only once.  This may break in the future.


#### Setup Conquest atomic basis set

In [15]:
basis = {
         'O' : { 'gen_basis'            : True,
                 'basis_size'           : 'small',
                 'pseudopotential_type' : 'hamann',
                 'xc'                   : 'PBE' },             
         'H' : { 'gen_basis'            : True,
                 'basis_size'           : 'small',
                 'pseudopotential_type' : 'hamann',
                 'xc'                   : 'PBE' },
         'C' : { 'gen_basis'            : True,
                 'basis_size'           : 'small',
                 'pseudopotential_type' : 'hamann',
                 'xc'                   : 'PBE' },         
          }

#### Setup calculation using Conquest as calculator

In [16]:
cutoff  =  90.0
kpoints = [1,1,1]
fxc     = 'PBE'

conquest_flags = {'IO.WriteOutToASEFile': True}

# Flags for atomic positions optimisation
conquest_flags.update({'AtomMove.TypeOfRun'   : 'sqnm',  # optimization algorithm
                       'AtomMove.MaxForceTol' :  5e-3,   # max Force component in Ha/bohr                       
                       'AtomMove.ReuseDM'     :  True,    
                       'AtomMove.AppendCoords':  True,
                       })    

calc = Conquest(directory      = working_directory,
                grid_cutoff    = cutoff,
                self_consistent= True,
                xc    = fxc,
                basis = basis,
                kpts  = kpoints,
                nspin = 1,
                **conquest_flags)

struct.calc = calc

input basis:
O {'gen_basis': False, 'basis_size': 'small', 'pseudopotential_type': 'hamann', 'xc': 'PBE'}
input basis:
H {'gen_basis': False, 'basis_size': 'small', 'pseudopotential_type': 'hamann', 'xc': 'PBE'}
input basis:
C {'gen_basis': False, 'basis_size': 'small', 'pseudopotential_type': 'hamann', 'xc': 'PBE'}


#### Launch structure optimization

In [17]:
dft_energy = struct.get_potential_energy()

Try to find H.ion in /Users/lioneltruflandier/CONQUEST-develop/pseudo-and-pao/H.ion... Not found
Try to find H.ion in /Users/lioneltruflandier/CONQUEST-develop/pseudo-and-pao/lib/H.ion... Not found
Try to find H.ion in /Users/lioneltruflandier/CONQUEST-develop/pseudo-and-pao/PBE/H/H.ion... Not found
Try to find H.ion in H.ion... Not found
Try to find H.ion in cq_example_phenol_optpos/H.ion... Found
## ConquestWarning ##
file: /Users/lioneltruflandier/Conquest-ase/ase-Conquest/ase/calculators/conquest.py , line: 283
>>>>  cq_example_phenol_optpos/H.ion copied into cq_example_phenol_optpos/H.ion
Try to find C.ion in /Users/lioneltruflandier/CONQUEST-develop/pseudo-and-pao/C.ion... Not found
Try to find C.ion in /Users/lioneltruflandier/CONQUEST-develop/pseudo-and-pao/lib/C.ion... Not found
Try to find C.ion in /Users/lioneltruflandier/CONQUEST-develop/pseudo-and-pao/PBE/C/C.ion... Not found
Try to find C.ion in C.ion... Not found
Try to find C.ion in cq_example_phenol_optpos/C.ion... Fou

#### Post-process `trajectory.xsf` file generated by Conquest and save `struct_last` in CIF and XSF format
- `trajectory.xsf` is processed in order to have a `axsf` movie 

> `cq_postproc_xsf`return an Atoms ASE object `struct_last`

In [18]:
traj_file='trajectory.xsf'
traj_file=working_directory+'/'+traj_file

if ( os.path.isfile(traj_file) ):
    # cq_postproc_xsf output last structure from xsf file
    struct_last = cq_postproc_xsf(traj_file,len(struct))
    
    struct_last.write(working_directory+'/output.cif')
    struct_last.write(working_directory+'/output.xsf')    
else:
    print('WARNING: {} not found !'.format(traj_file))


- or extract last structure from Conquest file `coord_next.dat` 
> you need to provide `atomic_order` as in the Conquest input file

In [ ]:
last_struct='coord_next.dat'
last_struct=working_directory+'/'+last_struct

if( os.path.isfile(last_struct) ):

    struct_last = read_conquest(last_struct, fractional=True, atomic_order=['H','C','O'])
    
    struct_last.write(working_directory+'/output_.cif')
    struct_last.write(working_directory+'/output_.xsf')
    
else:
    print('WARNING: {} not found !'.format(traj_file))

- run `xcrysden` to visualize the optimization movie (if possible)

In [ ]:
if ( cmd_vis['xcrysden'] and os.path.isfile(traj_file) ):
    subprocess.run(['xcrysden', '--axsf', traj_file])

#### Read Conquest output results and print the data

In [ ]:
struct.calc.read_results(struct)

cq_print_output_data(struct,2)

#### Extract and print main structural data from optimized structure `last_struct`

In [ ]:
print_struct_data(struct_last,verbose=1)

#### Extract Fermi energy, band gap and energy levels

In [ ]:
# Fermi energy as evaluated in Conquest code 'Fermi energy for spin'
fermi_energy = calc.get_fermi_level()
eigenvalues  = calc.get_eigenvalues()

print('Total electronic energy = {:14.6f} eV'.format(dft_energy))
print('Fermi energy            = {:14.6f} eV'.format(fermi_energy))
print_occ(eigenvalues,fermi_energy,0.01,method='aufbau',label='G')

# Evaluate band gap
gap, p1, p2  = bandgap(struct.calc,efermi=fermi_energy,direct=True)

## 